### Visualizing robot senses using ROS and Bokeh library

As you can see, robots publish many topics that could be interesting to see. 
Visualizing state of a sensor is a first step to programming using this sense or debugging if something goes wrong.

We will start with visualizing the state of the turtlesim through some cool techniques.

In [1]:
%env ROS_MASTER_URI=http://localhost:11311
import rospy
import roslaunch
import rosnode



env: ROS_MASTER_URI=http://localhost:11311


In [11]:
import os

os.system("rosrun ipython_robot_prototyping roomba_simulator.py &")
os.system("rosrun turtlesim turtlesim_node &")

0

With the fallowing commands we have started two ROS Nodes -- one is a familiar turtlebot simulator and the other is a simple control node for the turtle that turns turtle around when she is near the wall so that it does not crash.


What is also does is it calculates the distance of the turtle to the wall she is heading to.

Lets see all the turtle's topics

In [3]:
turtle_topics= tuple((topic_name,topic_type) 
                     for topic_name, topic_type in rospy.get_published_topics()
                     if "turtle" in topic_name)

print(turtle_topics)

(('/turtle1/distance', 'sensor_msgs/Range'), ('/turtle1/color_sensor', 'turtlesim/Color'), ('/turtle1/cmd_vel', 'geometry_msgs/Twist'), ('/turtle1/pose', 'turtlesim/Pose'))


There are couple of interesting topics there, like distance or pose. How we can visualise them?

In case of turtlesim it is easy to see what the distance or position are by just looking at it. That is nice but in case of real robots we are not only interested in the actual position and orientation but also on robots _sensor_ readings that is, robots sensors. The distance in /turtle1/distance is something calculated, but does it actally make sense? It is very important to have a way to visualise sensor data and luckyly, there are many ways in ROS to do it:

 - The easiest way is to use _rostopic echo_ command. You just put, for example rostopic echo /turtle1/distance to see the distance values. Try to do it now.
 
 - The second option is to use another ROS command -- rqt_plot.


In [19]:
os.system("rqt_plot /turtle1/distance/range")

0

The command is nice as it automaticly scales a plot to fit the changing data. It also allows you to add different topics through graphical user interface or by putting different topic names when starting the tool( i.e. rqt_prot /turtle1/distance/range /turtle1/pose/x ). There is also a settings button in the right top corner to further set things up

 - It would be nice though to see the things directely in the Ipython notebook, to be able to see how the things change when we play around. We will show you couple of ways to do that

### Ipython widgets

The first way, especially if there is a single changing value is to use IPython slider but as an output view.

The easiest way it to change the sliders __value__ parameter in subscriber callback function.

First let's create a slider. Because range is a float, we will be using a float slider. 

In [15]:
from ipywidgets import FloatSlider
from IPython.display import display
distance_slider = FloatSlider(
    value=7.5,
    min=0,
    max=10.0,
    step=0.1,
    description='The distance to the wall',
)

distance_slider

Now, let's create a subscriber with a callback function that changes value of the slider. 



In [5]:
rospy.init_node("distance_reader")

In [20]:
from sensor_msgs.msg import Range
def read_distance_show_on_slider(range_msg,slider):
    ''' The callback function that reads the range message from the turtle and presents it on a slider'''

    slider.value=range_msg.range


distance_subscriber= rospy.Subscriber('turtle1/distance',
                                      Range, 
                                      read_distance_show_on_slider,callback_args=distance_slider)

In [24]:
### this will stop updating the slider
distance_subscriber.unregister()

We can see that the slider created before runs on its own! Now it is much easier to understand what the robot "sees" when running around.

Notice that our callback function has a second argument, to have room for some universality. We pass value to this argument by putting it to callback_args parameter when defining the Subscriber object

### Excercise:

There is also a control message from our "roomba simulator". Try to read the control message to see how they synchronize. Create two new sliders one for cmd vel linear x and the second for cmd vel angular z.

Put the 3 sliders one below the other to see how these values are synchronized

In [12]:
linear_x_slider = FloatSlider(
    value=0,
    min=-2,
    max=2,
    step=0.1,
    description='The forward speed',
)

angular_z_slider = FloatSlider(
    value=0,
    min=-10,
    max=10,
    step=0.1,
    description='The angular speed',
)


In [ ]:
### define the callback function

def read_x_z_show_on_slider(range_msg):
    
    linear_x_slider.value=...
    angular_z_slider.value=...
    
### define here the two subscribe objects

linear_x_subscriber = rospy.Subscriber('turtle1/distance')
angular_z_subscriber = rospy.Subscriber


In [18]:
### we can display all the sliders here

display(distance_slider)
display(linear_x_slider)
display(angular_z_slider)

While this approach is ok, you can see that we are mixing the display of the data (slider) with some logic -- the things we do when reading the values. We can do it in nicer way using objects. Our object will have _traits_ -- special elements that will we will know when they change. 

In [23]:
import traitlets
class Gather_Data(traitlets.HasTraits):
    '''A class that gathers information from two topics'''
    
    # here we define a 3 traits that we will fallow
    distance=traitlets.Float()
    lin_speed=traitlets.Float()
    ang_speed=traitlets.Float()
    
    def __init__(self):
        pass